In [ ]:
!pip install nb_black
!pip install torch

In [4]:
%load_ext lab_black

In [63]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
from transformers import EncoderDecoderModel, BertTokenizerFast
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

import datasets
import transformers
import os
import pickle
import random

import pandas as pd
import datasets
from datasets import Dataset, DatasetDict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
# model = RobertaModel.from_pretrained("microsoft/codebert-base")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

# model.to(device)

# tokenizer.bos_token = tokenizer.cls_token
# tokenizer.eos_token = tokenizer.sep_token

loading file vocab.txt from cache at /home/RDC/zinovyee.hub/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file tokenizer.json from cache at /home/RDC/zinovyee.hub/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/RDC/zinovyee.hub/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /home/RDC/zinovyee.hub/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classi

In [82]:
with open(
    os.path.join("../data", "repos_as_pandas_metainfo_20230126.pkl"), "rb"
) as file:
    repos = pickle.load(file)
repos = repos[repos.type_script == "py"]


def standardize_types(x):
    if isinstance(x, list):
        return ",".join(x)
    else:
        return x


repos.meta_keywords = repos.meta_keywords.apply(standardize_types)

In [96]:
batch_size = 4  # change to 16 for full training
encoder_max_length = 512
decoder_max_length = 6


def process_data_to_model_inputs(batch):

    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["meta_desc"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
    )
    outputs = tokenizer(
        batch["meta_keywords"],
        padding="max_length",
        truncation=True,
        max_length=decoder_max_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()

    # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [97]:
repos["key_len"] = repos.meta_keywords.apply(len)
repos["code_len"] = repos.meta_desc.apply(len)

repos = repos[repos["code_len"] > 5]
repos = repos[repos["key_len"] > 5]

repos["repo_name"] = repos.q_name.str.split("/", expand=True)[0]
# repos["meta_desc"] = repos["meta_desc"].apply(lambda x: [line.decode() for line in x])
# repos["meta_desc"] = repos["meta_desc"].apply(lambda x: " ".join(x))

In [98]:
ids = list(range(repos.repo_name.nunique()))
train_idx = random.sample(ids, 100)
test_idx = [id for id in ids if id not in train_idx]

train_repos = repos["repo_name"].unique()[train_idx]
test_repos = repos["repo_name"].unique()[test_idx]

train_data = Dataset.from_pandas(
    repos.loc[repos.repo_name.isin(train_repos), ["meta_desc", "meta_keywords"]]
)
test_data = Dataset.from_pandas(
    repos.loc[repos.repo_name.isin(test_repos), ["meta_desc", "meta_keywords"]]
)

In [99]:
val_data = test_data

In [100]:
# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
# rain_data = train_data.select(range(32))

train_data = train_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["meta_desc", "meta_keywords"],
)
train_data.set_format(
    type="torch",
    columns=[
        "input_ids",
        "attention_mask",
        "decoder_input_ids",
        "decoder_attention_mask",
        "labels",
    ],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data = val_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["meta_desc", "meta_keywords"],
)
val_data.set_format(
    type="torch",
    columns=[
        "input_ids",
        "attention_mask",
        "decoder_input_ids",
        "decoder_attention_mask",
        "labels",
    ],
)

  0%|          | 0/191 [00:00<?, ?ba/s]

  0%|          | 0/80 [00:00<?, ?ba/s]

decoder = BertGenerationDecoder.from_pretrained(
    "bert-large-uncased", add_cross_attention=True, is_decoder=True, bos_token_id=101, eos_token_id=102
)

In [112]:
from transformers import BertGenerationDecoder

In [113]:
decoder = BertGenerationDecoder.from_pretrained(
    "bert-large-uncased",
    add_cross_attention=True,
    is_decoder=True,
    bos_token_id=101,
    eos_token_id=102,
)

loading configuration file config.json from cache at /home/RDC/zinovyee.hub/.cache/huggingface/hub/models--bert-large-uncased/snapshots/80792f8e8216b29f3c846b653a0ff0a37c210431/config.json
You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
Model config BertGenerationConfig {
  "add_cross_attention": true,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 101,
  "eos_token_id": 102,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert-generation",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_v

loading weights file pytorch_model.bin from cache at /home/RDC/zinovyee.hub/.cache/huggingface/hub/models--bert-large-uncased/snapshots/80792f8e8216b29f3c846b653a0ff0a37c210431/pytorch_model.bin
Generate config GenerationConfig {
  "bos_token_id": 101,
  "eos_token_id": 102,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertGenerationDecoder: ['bert.embeddings.token_type_embeddings.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertGenerationDecoder from the checkpoint of a model trained on another task or with another architecture (e.g. ini

In [115]:
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained(
    "bert-base-uncased",
    decoder
    # "microsoft/codebert-base", "roberta-base"
)

# set special tokens
bert2bert.config.decoder_start_token_id = tokenizer.bos_token_id
bert2bert.config.eos_token_id = tokenizer.eos_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
bert2bert.config.vocab_size = bert2bert.config.decoder.vocab_size
bert2bert.config.max_length = 6
bert2bert.config.min_length = 4
bert2bert.config.no_repeat_ngram_size = 2
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4

loading configuration file config.json from cache at /home/RDC/zinovyee.hub/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/RDC/zinovyee.hub/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4

OSError: Can't load the configuration of 'BertGenerationDecoder(
  (bert): BertGenerationEncoder(
    (embeddings): BertGenerationEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertGenerationLayer(
          (attention): BertGenerationAttention(
            (self): BertGenerationSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertGenerationSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
     

In [116]:
transformers.__version__

'4.26.0'

In [102]:
# load rouge for validation
rouge = datasets.load_metric("rouge")


def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge1_precision": round(rouge_output.precision, 4),
        "rouge1_recall": round(rouge_output.recall, 4),
        "rouge1_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [104]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_steps=2,  # set to 1000 for full training
    save_steps=500,  # set to 500 for full training
    eval_steps=8000,  # set to 8000 for full training
    warmup_steps=2000,  # set to 2000 for full training
    # max_steps=16, # delete for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=288, training_loss=0.053289004200552076, metrics={'train_runtime': 88.8867, 'train_samples_per_second': 25.752, 'train_steps_per_second': 3.24, 'total_flos': 1404196435998720.0, 'train_loss': 0.053289004200552076, 'epoch': 3.0})

In [107]:
import datasets
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_pretrained("./checkpoint-16")
model.to("cuda")

loading file vocab.txt from cache at /home/RDC/zinovyee.hub/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/RDC/zinovyee.hub/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /home/RDC/zinovyee.hub/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_siz

EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [108]:
# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# test_data = test_data.select(range(16))

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(
        batch["meta_desc"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch


results = test_data.map(
    generate_summary,
    batched=True,
    batch_size=batch_size,
    remove_columns=["meta_desc"],
)

pred_str = results["pred"]
label_str = results["meta_keywords"]

rouge_output = rouge.compute(
    predictions=pred_str, references=label_str, rouge_types=["rouge2"]
)["rouge2"].mid

print(rouge_output)

  0%|          | 0/20 [00:00<?, ?ba/s]

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 142,
  "min_length": 56,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 142,
  "min_length": 56,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
 

Score(precision=0.0, recall=0.0, fmeasure=0.0)


In [109]:
pred_str

['follow follow follow following follow follow follows follow follows follows follows follow following follows follow follow followed follow follows following follows follows following follow following following follows following following follow follows followed follow follow contact follow follow point follow follow meet follow follow lead follow follow cross follow follow meeting follow follow along follow follow view follow follow body follow follow match follow follow like follow follow focus follow follow beads follow follow base follow follow stream follow follow receive follow follow study follow follow step follow follow influence follow follow reading follow follow take follow follow cane follow follow protocol follow follow post follow followt follow follow primary follow follow forward follow follow move follow follow stage follow follow read follow follow mic follow follow play follow follow classic follow follow flow follow follow goal follow follow guide follow',
 'follo